## Heart Disease 
Questions:
1. What are the dimensions of the matrices you will use to represent your model (inputs, parameters, and
outputs)? How will you integrate the concept of mini-batch training?
2. How to check whether or not you should keep training your model? 

## Step 1: Dataset Loading and Preprocessing

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Load dataset
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
#infromation 
df.info

<bound method DataFrame.info of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1    

In [7]:
#data column types
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [8]:
#null values
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [9]:
#column names
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [11]:
# Split data into features (X) and target (y)
X = df.drop(columns=['target'])
y = df['target'].values

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Step 2: Initialize Weights and Biases

In [12]:
# Define dimensions
input_dim = X_train.shape[1]  # 13 features
hidden_1_dim = 6  # First hidden layer
hidden_2_dim = 4  # Second hidden layer
output_dim = 1  # Output layer (binary classification)

# Initialize weights and biases
np.random.seed(42)  # For reproducibility
W1 = np.random.randn(input_dim, hidden_1_dim) * 0.01
b1 = np.zeros((1, hidden_1_dim))

W2 = np.random.randn(hidden_1_dim, hidden_2_dim) * 0.01
b2 = np.zeros((1, hidden_2_dim))

W3 = np.random.randn(hidden_2_dim, output_dim) * 0.01
b3 = np.zeros((1, output_dim))

## Step 3: Define Activation Functions

In [13]:
# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Derivative of sigmoid (for backpropagation)
def sigmoid_derivative(z):
    return z * (1 - z)

## Step 4: Forward Propagation

In [14]:
# Forward propagation
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)

    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)
    
    return A1, A2, A3

## Step 5: Compute Loss

In [15]:
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -1/m * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

## Step 6: Backpropagation

In [16]:
def backpropagation(X, y, A1, A2, A3, W2, W3):
    m = X.shape[0]
    
    # Output layer error
    dZ3 = A3 - y.reshape(-1, 1)
    dW3 = 1/m * np.dot(A2.T, dZ3)
    db3 = 1/m * np.sum(dZ3, axis=0, keepdims=True)

    # Second hidden layer error
    dZ2 = np.dot(dZ3, W3.T) * sigmoid_derivative(A2)
    dW2 = 1/m * np.dot(A1.T, dZ2)
    db2 = 1/m * np.sum(dZ2, axis=0, keepdims=True)

    # First hidden layer error
    dZ1 = np.dot(dZ2, W2.T) * sigmoid_derivative(A1)
    dW1 = 1/m * np.dot(X.T, dZ1)
    db1 = 1/m * np.sum(dZ1, axis=0, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3

## Step 7: Update Parameters

In [17]:
def update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    return W1, b1, W2, b2, W3, b3

## Step 8: Training Loop

In [18]:
# Set parameters
epochs = 1000
learning_rate = 0.01

# Training loop
for epoch in range(epochs):
    # Forward pass
    A1, A2, A3 = forward_propagation(X_train, W1, b1, W2, b2, W3, b3)
    
    # Compute loss
    loss = compute_loss(y_train, A3)
    
    # Backward pass
    dW1, db1, dW2, db2, dW3, db3 = backpropagation(X_train, y_train, A1, A2, A3, W2, W3)
    
    # Update parameters
    W1, b1, W2, b2, W3, b3 = update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

Epoch 0, Loss: 167.79604829051962
Epoch 100, Loss: 167.0093074561381
Epoch 200, Loss: 166.71944208912322
Epoch 300, Loss: 166.6124304744887
Epoch 400, Loss: 166.5728574972328
Epoch 500, Loss: 166.55820566679859
Epoch 600, Loss: 166.5527764182763
Epoch 700, Loss: 166.55076350971254
Epoch 800, Loss: 166.5500169401133
Epoch 900, Loss: 166.5497399681572


## Step 9: Prediction and Evaluation

In [19]:
def predict(X, W1, b1, W2, b2, W3, b3):
    _, _, A3 = forward_propagation(X, W1, b1, W2, b2, W3, b3)
    return (A3 > 0.5).astype(int)

# Predictions
y_pred = predict(X_test, W1, b1, W2, b2, W3, b3)

# Accuracy
accuracy = np.mean(y_pred == y_test.reshape(-1, 1))
print(f"Test accuracy: {accuracy * 100}%")

Test accuracy: 52.459016393442624%
